Some Popular Tasks Regarding Text Processing:

* **Language Translation** - Translation of a sentence from one language to another
* **Sentiment Analysis** - To determine whether the sentiment towards any topic or product is positive, negative or neutral, based on a corpus of text
* **Spam Filtering** - To detect unsolicited and unwanted email/messages

In this notebook, we'll discuss the steps involved in text processing.

# Data Preprocessing
***
The data preprocessing steps could include:
* **Tokenization** - converting sentences to words
* Removing unnecessary punctuation and tags
* Removing stop words
* Stemming - Removing inflection via dropping unnecessary characters (usually a suffix)
* Lemmatization - Removing inflection by determining the part of speech and utilized a detailed database of the language

Stemming is the poor man's lemmatization

```text
The stemmed form of studies is: studi
The stemmed form of studying is: study

The lemmatized form of studies is: study
The lemmatized form of studying is: study
```

We can use the `nltk` library to do a lot of text preprocessing:
## Tokenization
***

In [1]:
import nltk
from nltk.tokenize import word_tokenize

text = "The quick brown fox jumps over the lazy dog"
tokens = word_tokenize( text )
print(tokens)

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


## Removing Stopwords
***
We can use `nltk` to remove stop words (words containing no semantic value)

If you need to download the stopwords for `nltk`, you need to run:
```python
nltk.download("stopwords")
```

In [2]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
tokens = [token for token in tokens if token not in stop_words]
print(tokens)

['The', 'quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']


## Stemming
***
`nltk` also provides several stemmer interfaces like Porter stemmer, Lancaster stemmer and snowball stemmer:

In [3]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
stems = []
for token in tokens:
    stems.append(stemmer.stem(token))
print(stems)

['the', 'quick', 'brown', 'fox', 'jump', 'lazi', 'dog']


# Feature Extraction
***
In text processing, words represent discrete, categorical features. How do we encode this categorical data in a way that is ready to be used by the algorithms? One of the simples techniques is the **bag of words** featurization.

## Bag of Words (BOW)
***
We make a **list of unique words** in the corpus, called the **vocabulary**. Then each word in the vocabulary gets its own basis vector. Then a sentence, or each document in the corpus is represented by a vector that is equal to the vector sums of the basis vectors for the words appearing in that sentence. 

This leads us to the **Term Frequency-Inverse Document Frequency (TF-IDF)** technique:

## TF-IDF
***
First, let us clarify what is meant by document. A corpus is made up of many documents. So if your corpus is a set of tweets, each document is a tweet. So we can ask for the word vector of an entire tweet. The word vector of an entire tweet would be some function of the word vectors of its constituent words.

$$\textrm{Term Frequency (TF)} = \frac{\textrm{number of times token appears in single document}}{\textrm{number of tokens in single document}}$$

$$\textrm{Inverse Document Frequency (IDF)} = \log\left(\frac{\textrm{total number documents}}{\textrm{number of documents this token appears in}}\right)$$

$$\textrm{TF-IDF} = (\textrm{TF})(\textrm{IDF})$$

Here is an example of calculating the TF-IDF of a term in a document:

```text
tweet_one = "This is a beautiful beautiful day day day day day"
tweet_two = "This is a beautiful night night"
```

Then we would have that:
```text
TF("beautiful",tweet_one) = 2/10
TF("day", tweet_one) = 5/10
IDF("beautiful") = log(2/2) = 0
IDF("day") = log(2/1) = 0.3

TF_IDF("beautiful", tweet_one) = (2/10)(0)=0
TF_IDF("day", tweet_one) = (5/10)(0.30) = 0.15
```

So we see that for the first tweet, the TF-IDF method heavily penalizes the word "beautiful", but assigns greater weight to "day". "beautiful" gives no power of resolution because it appears in both documents. "day" is an important word for `tweet_one` in the context of the entire corpus (it only appears in one of the two tweets). 

`scikit-learn` provides efficient tools for computing the TF-IDF of a corpus.

One of the major disadvantages of using the bag of words featurization is that it discards all information contained in the word order of the vectors. 

To solve this problem, we use an approach called **Word Embedding**
## Word Embedding
***
A word embedding is a numerical representation of text where words that have similar semantic meaning are geometrically closer in their semantic vector space. 

### Word2Vec
***
Word2Vec takes a corpus of text and produces a vector space that has the property that vectors that are geometrically close also share common semantic contexts in the corpus. 

### Glove
***
The **Global Vectors for Word Representation** algorithm is an extension to the Word2Vec model. GloVe constructs a co-occurrence matrix on the whole text corpus. The entries of this matrix are the probabilities that a given token appears in the common of every other word in the vocabulary. This assumes the **distributional hypothesis**.

# Choosing ML Algorithms
***
Classical ML approaches such as Naive bayes or Support Vector machines for spam filtering are very popular. However, Deep Learning techniques, combined with deep-learned word embeddings are taking the NLP stage by storm.

# Example IMDB Movie Review Sentiment Analyzer
***
We will now build a sentiment analyzer over the IMDB movie review dataset. 

We will be performing binary classification (negative or positive reviews).

Our dataset can be downloaded [here](http://ai.stanford.edu/~amaas/data/sentiment/).

The dataset contains 25000 training reviews and 25000 testing reviews. 

## Loading the Data
***
The data is separated into test/pos, test/neg, train/pos, train/neg folders containing text files for each review. We need to load all of these reviews into a dataframe so we can do stuff with it. 

In [4]:
import pandas as pd
import logging
import os

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s",\
                   level=logging.INFO)

folder = "data/imdb_movie_reviews/"
labels = {'pos':1, 'neg':0}

data = pd.DataFrame()
i = 0
for dataset in ["test","train"]:
    for polarity in ["pos","neg"]:
        preliminary_path = os.path.join(folder, dataset, polarity)
        for file in os.listdir(preliminary_path):
            full_file_name = os.path.join(preliminary_path, file)
            with open(full_file_name, 'r', encoding='utf-8') as fileHandle:
                review_text = fileHandle.read()
                review = pd.DataFrame([{"review":review_text,"sentiment":labels[polarity]}])
            i += 1
            data = data.append(review, ignore_index=True)
            if (i%10000==0):
                logging.info("read {} reviews".format(i))

2019-08-12 17:14:54,166 : INFO : read 10000 reviews
2019-08-12 17:15:08,754 : INFO : read 20000 reviews
2019-08-12 17:15:25,529 : INFO : read 30000 reviews
2019-08-12 17:15:45,038 : INFO : read 40000 reviews
2019-08-12 17:16:08,094 : INFO : read 50000 reviews


In [5]:
data

,review,sentiment
0,Original Movie lovers can actually love this s...,1
1,Peak Practice was a British drama series about...,1
2,My wife is a mental health therapist and we wa...,1
3,A fantastic film featuring great Aussie talent...,1
4,Despite all it's trappings of style and cinema...,1
5,I finally watched the third film in Mehta's tr...,1
6,I saw this movie tonight in a preview showing ...,1
7,Lackawana Blues An impressive HBO movie about ...,1
8,Aaron Sorking raises the same questions as Sha...,1
9,Martin Ritt seems to be a director who was alw...,1


## Look at the Frequency of Top Words
***
We can get the frequency distribution of the words in the text by utilizing the `nltk.FreqDist()` function. This function lists the top words used in the text. We can look at the top 50 most frequent words in the text: As you can see, we have a lot of stop words in there. 

In [6]:
# all of the words in the corpus concatenated
reviews = data.review.str.cat(sep=" ")
tokens = word_tokenize(reviews)
vocabulary = set(tokens)
print("Number of unique words in vocabulary: {}".format(len(vocabulary)))
frequency_dist = nltk.FreqDist(tokens)
print(sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=True)[:50])

Number of unique words in vocabulary: 198758
['the', ',', '.', 'a', 'and', 'of', 'to', 'is', '/', '>', '<', 'br', 'in', 'I', 'it', 'that', "'s", 'this', 'was', 'The', 'as', 'with', 'movie', 'for', 'film', ')', '(', 'but', "''", "n't", '``', 'on', 'you', 'are', 'not', 'have', 'his', 'be', '!', 'he', 'one', 'at', 'by', 'an', 'all', 'who', 'they', 'from', 'like', 'It']


## Remove Stopwords
***
Now we shall remove stopwords to further clean up the text corpus:

In [7]:
tokens = [word for word in tokens if word not in stop_words]
frequency_dist = nltk.FreqDist(tokens)
print(sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=True)[:50])

[',', '.', '/', '>', '<', 'br', 'I', "'s", 'The', 'movie', 'film', ')', '(', "''", "n't", '``', '!', 'one', 'like', 'It', '?', 'This', 'good', 'would', '...', 'time', 'really', 'see', 'even', 'story', "'", ':', '-', 'much', 'could', 'get', 'people', 'bad', 'great', 'well', 'first', 'made', 'also', 'make', 'way', 'movies', 'But', 'think', 'characters', 'character']


## Wordcloud Visualization
***
Now we can make a wordcloud visualization.

In [8]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud().generate_from_frequencies(frequency_dist)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

<Figure size 640x480 with 1 Axes>

## Building a Classifier
***
Now we can divide the dataset into a test and training set of 25000 reviews each, appropriately stratified by label:

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data.review,data.sentiment,stratify=data.sentiment,\
                                                   random_state=2019,test_size=0.2)

## Featurization
***
Now we need to actually convert our text into a numerical vector representation. We will use TF IDF from scikit-learn here

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)

print(train_vectors.shape, test_vectors.shape)

(40000, 92929) (10000, 92929)


## Black Box Model
***
Now we will use the black box naive bayes model from scikit-learn to create a baseline model for this problem:

In [11]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(train_vectors, y_train)

## Metrics
***
Now we use the basic accuracy as the metric for our classification, because our classes are balanced.

In [12]:
from sklearn.metrics import classification_report
predictions = clf.predict(test_vectors)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      5000
           1       0.88      0.84      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000



# An Improved Sentiment Classifier using Deep Learning
***

The architecture for the deep learning text classification model generally consists of the following connected components:

Text(X) -> Embedding -> Deep Network (LSTM/GRU) -> Fully Connected (Dense) -> Output Layer (Softmax) -> Sentiment (y)

## Clean the Text for Embedding Layer
***
The word embeddings can be learned while training a neural network on the classification problem. Before it can be presented to the network, the text data is encoded so that each word is represented by a unique integer. This can be performed using the **Tokenizer API** provided by **Keras**. 

We also add padding to make all of the vectors the same length. 

In [13]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.review.values)

max_length = data.review.apply(lambda review: len(review.split())).max(axis=0)
print("Maximum length of any review: {}".format(max_length))

# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1

train_tokens = tokenizer.texts_to_sequences(x_train.values)
test_tokens = tokenizer.texts_to_sequences(x_test.values)

x_train_padded = pad_sequences(train_tokens, maxlen=max_length, padding="post")
x_test_padded = pad_sequences(test_tokens, maxlen=max_length, padding="post")

Maximum length of any review: 2470


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

# dimension of semantic word vector space
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="adam", metrics=["accuracy"])
print(model.summary())

Using TensorFlow backend.
W0812 17:18:27.048817 140149158770496 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 17:18:27.069983 140149158770496 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 17:18:27.072511 140149158770496 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 17:18:27.143563 140149158770496 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Ple

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2470, 100)         12425300  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,438,101
Trainable params: 12,438,101
Non-trainable params: 0
_________________________________________________________________
None


Now we can fit the model using our training data and some hyperparameters.

In [ ]:
model.fit(x_train_padded, y_train.values, batch_size=400, epochs=5, \
          validation_data=(x_test_padded,y_test.values), verbose=2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/25


Now we test the model on some custom reviews that are clearly negative or positive to see how good it is at predicting the sentiment of each review.

In [ ]:
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6,
               test_sample_7, test_sample_8]

test_samples_tokenized = tokenizer.texts_to_sequences(test_samples)
test_samples_padded = pad_sequences(test_samples_tokenized, maxlen=max_length)

print(model.predict(x=test_samples_padded))

## Train a Word2Vec Embedding
***
Now we  will try to separately learn the word embeddings and then pass them to the embedding layer. This also allows us to use per-trained word embeddings. We will use `gensim` here. 

The **first step** is to prepare the corpus for the learning of the embeddings by **tokenization**, **removing punctuation**, **removing stop words**, etc. The word2vec algorithm processes documents sentence by sentence.

In [23]:
import string

punctuation_pattern = "[" + string.punctuation + "]"
re.sub(pattern,"",my_string)

cleaned_reviews = []
reviews = data.review.values.tolist()

for review in reviews:
    # create tokens for this review
    tokens = word_tokenize(review)
    # convert all words to lowercase
    tokens = [word.lower() for word in tokens]
    # convert punctuation to empty space
    tokens_less_punc = [re.sub(punctuation_pattern,"",token) for token in tokens]
    # remove non alphabetic tokens
    tokens = list(filter(str.isalpha, tokens_less_punc))
    tokens = list(filter(lambda word:word not in stop_words, tokens))
    cleaned_reviews.append(tokens)

In [25]:
print(cleaned_reviews[0])

['original', 'movie', 'lovers', 'actually', 'love', 'show', 'stop', 'complaining', 'time', 'br', 'br', 'emperor', 'new', 'school', 'brings', 'old', 'jokes', 'movie', 'like', 'pulling', 'lever', 'yzma', 'lab', 'kuzco', 'pausing', 'episode', 'since', 'kids', 'show', 'classic', 'right', 'places', 'even', 'though', 'style', 'much', 'simple', 'animation', 'characters', 'keeps', 'personalities', 'well', 'surprised', 'actually', 'eartha', 'kitt', 'makes', 'excellent', 'voice', 'acting', 'yzma', 'jp', 'manoux', 'wonderful', 'job', 'kuzco', 'voice', 'instead', 'david', 'spade', 'played', 'kuzco', 'movie', 'great', 'plots', 'hilarious', 'moments', 'kuzco', 'amazing', 'looks', 'makes', 'show', 'worth', 'watching', 'stop', 'complaining', 'everything']


Now we can use `gensim` to find our embeddings. The parameters needed for our Word2Vec model are:

* `sentences` - a list of sentences
* `size` - the number of dimensions for word vectors
* `min_count` - minimum frequency for a word to be included
* `window` - only terms that occur within a window neighborhood of a term in a sentence are associated
* `workers` - number of threads used in training parallelization to speed up training

In [28]:
import gensim

gensim_model = gensim.models.Word2Vec( sentences=cleaned_reviews, \
                                      size=EMBEDDING_DIM, window=5, workers=8, min_count=1)

I0812 16:19:34.207339 139901176153920 word2vec.py:1145] collecting all words and their counts
I0812 16:19:34.208136 139901176153920 word2vec.py:1163] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
I0812 16:19:34.405547 139901176153920 word2vec.py:1163] PROGRESS: at sentence #10000, processed 1214193 words, keeping 57422 word types
I0812 16:19:34.596872 139901176153920 word2vec.py:1163] PROGRESS: at sentence #20000, processed 2418958 words, keeping 81729 word types
I0812 16:19:34.814720 139901176153920 word2vec.py:1163] PROGRESS: at sentence #30000, processed 3641415 words, keeping 101927 word types
I0812 16:19:35.047518 139901176153920 word2vec.py:1163] PROGRESS: at sentence #40000, processed 4890500 words, keeping 119311 word types
I0812 16:19:35.301706 139901176153920 word2vec.py:1175] collected 134087 word types from a corpus of 6106047 raw words and 50000 sentences
I0812 16:19:35.302377 139901176153920 word2vec.py:1208] Loading a fresh vocabulary
I0812 16:19:35.7

I0812 16:19:55.582199 139901176153920 base_any2vec.py:178] worker thread finished; awaiting finish of 1 more threads
I0812 16:19:55.585205 139901176153920 base_any2vec.py:178] worker thread finished; awaiting finish of 0 more threads
I0812 16:19:55.585967 139901176153920 base_any2vec.py:664] EPOCH - 4 : training on 6106047 raw words (5692093 effective words) took 4.4s, 1279415 effective words/s
I0812 16:19:56.603751 139901176153920 base_any2vec.py:650] EPOCH 5 - PROGRESS: at 21.01% examples, 1182179 words/s, in_qsize 15, out_qsize 0
I0812 16:19:57.616921 139901176153920 base_any2vec.py:650] EPOCH 5 - PROGRESS: at 42.99% examples, 1196022 words/s, in_qsize 16, out_qsize 1
I0812 16:19:58.618730 139901176153920 base_any2vec.py:650] EPOCH 5 - PROGRESS: at 65.83% examples, 1237257 words/s, in_qsize 15, out_qsize 0
I0812 16:19:59.619908 139901176153920 base_any2vec.py:650] EPOCH 5 - PROGRESS: at 88.44% examples, 1250632 words/s, in_qsize 15, out_qsize 0
I0812 16:20:00.100283 139901176153920 

In [29]:
words = list(gensim_model.wv.vocab)
print("Vocabulary size: {}".format(len(words)))

Vocabulary size: 134087


Now we can take a peek at some of the word embeddings learned from the review dataset:

In [30]:
gensim_model.wv.most_similar("horrible")

I0812 16:20:51.072980 139901176153920 keyedvectors.py:1042] precomputing L2-norms of word weight vectors


[('terrible', 0.9216718673706055),
 ('awful', 0.8762881755828857),
 ('sucks', 0.7772316336631775),
 ('horrendous', 0.769995391368866),
 ('atrocious', 0.7531569600105286),
 ('pathetic', 0.7523128986358643),
 ('dreadful', 0.7506479024887085),
 ('horrid', 0.742202877998352),
 ('lousy', 0.7384706139564514),
 ('laughable', 0.7182028889656067)]

Let's try some of the math Word2Vec is famous for: `woman+king-man=?`

In [31]:
gensim_model.wv.most_similar_cosmul(positive=["woman","king"], negative=["man"])

[('princess', 0.8810762166976929),
 ('romeo', 0.8673413991928101),
 ('prince', 0.8594143390655518),
 ('bride', 0.8522542119026184),
 ('queen', 0.850104808807373),
 ('onionpeeling', 0.8499622941017151),
 ('godmother', 0.8485676646232605),
 ('ammanda', 0.8458809852600098),
 ('furst', 0.8409291505813599),
 ('juliet', 0.8385201096534729)]

Next step is to use the word embeddings directly in the embedding layer in the sentiment classification model. We save the model to be used later:

The first column of each row is the word in plaintext, and the rest of the columns are the components of the word vector

In [33]:
filename = os.path.join("data/","imdb_embedding_word2vec.txt")
gensim_model.wv.save_word2vec_format(filename,binary=False)

I0812 16:23:18.838003 139901176153920 utils_any2vec.py:109] storing 134087x100 projection weights into data/imdb_embedding_word2vec.txt
/home/joseph/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Now we extract the word embeddings from the stored file:

In [34]:
embeddings_index = {}
with open(os.path.join("data/","imdb_embedding_word2vec.txt"), encoding="utf-8") as file:
    for line in file:
        values = line.split()
        # first column is the word
        word = values[0]
        # the rest of the columns in this row are the word vector components
        coefs = np.asarray(values[1:])
        embeddings_index[word] = coefs

The value of any entry in this dictionary is a list of the weights:

In [36]:
embeddings_index["br"]

array(['-0.06289237', '-4.1104317', '-0.7192977', '0.3467713',
       '0.15422522', '1.0290797', '0.29853466', '-0.4629418',
       '-0.28074723', '-0.52995634', '-0.48216525', '-0.63887423',
       '-0.61678463', '0.043714996', '1.1465778', '-0.0986907',
       '-0.017181644', '-1.1504191', '-0.7751319', '-1.192805',
       '0.1896599', '-0.78604263', '1.1917669', '0.14878462',
       '-0.07003895', '0.15610346', '-0.6093981', '1.3712231',
       '0.8185536', '-0.3403442', '-0.13370019', '-0.06265738',
       '0.5446949', '-0.68191046', '1.0355029', '-0.2035864',
       '-0.24064294', '-0.74360555', '0.56173146', '-1.7993212',
       '-1.6746668', '1.3080883', '1.2252704', '-1.5737292', '-1.6902249',
       '-0.3025173', '0.36642084', '0.4582488', '-0.45711353',
       '-1.1066535', '0.45047405', '1.3150895', '-0.09814261',
       '-0.5541568', '-1.140533', '-0.35467556', '0.5681579',
       '-0.41108182', '-1.7969784', '0.26176956', '-0.21563247',
       '-0.29490942', '0.012024802',

Now we convert the word embedding into a tokenized vector. Recall that the review documents are integer encoded prior to passing them tot he embedding layer. THe integer maps the index of a specific vector in the embedding layer. Therefore, it is important that we lay the vectors out in the Embedding layer such that the encoded words map the correct vector. 

In [39]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_reviews)
sequences = tokenizer.texts_to_sequences(cleaned_reviews)

word_index = tokenizer.word_index
print("Found {} unique tokens".format(len(word_index)))

review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment = data.sentiment.values

print("Shape of reviews tensor: {}".format(review_pad.shape))
print("Shape of sentiment tensor: {}".format(sentiment.shape))

Found 134087 unique tokens
Shape of reviews tensor: (50000, 2470)
Shape of sentiment tensor: (50000,)


Now we map the embeddings from the loaded word2vec model for each word to the `tokenizer.word_index` vocabulary and we create a matrix of word vectors:

In [41]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word) # same as __getitem__ but does checking
    if embedding_vector is not None: # so that we can do this line
        embedding_matrix[i] = embedding_vector
    # else that row of embedding matrix will be all zeros cause we initialized it so

Now we can train the embedded vector to be used directly in the embedding layer. In the below code, the only change from the previous model is that we are using `embedding_matrix` as input to the embedding layer and we are setting `trainable=False` because the embedding is already learned.

In [42]:
from keras.initializers import Constant
model = Sequential()
model.add(Embedding(num_words, \
                    EMBEDDING_DIM, \
                    embeddings_initializer=Constant(embedding_matrix),\
                    input_length=max_length, \
                    trainable=False))

model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="adam", metrics=["accuracy"])
print(model.summary())

W0812 16:39:54.945634 139901176153920 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 16:39:55.481924 139901176153920 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 16:39:55.804842 139901176153920 deprecation_wrapper.py:119] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0812 16:39:55.812100 139901176153920 deprecation.py:506] From /home/joseph/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is dep

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2470, 100)         13408800  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 13,421,601
Trainable params: 12,801
Non-trainable params: 13,408,800
_________________________________________________________________
None


Now if you notice, we've cut down the `trainable_params` to be much lower than the total number of params. Therefore, since the model uses pre-trained word embeddings, we have few trainable parameters and it should train faster. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(review_pad,sentiment,stratify=sentiment,\
                                                   random_state=2019,test_size=0.2)

In [ ]:
print("Shape of x_train: {}".format(x_train.shape))
print("Shape of y_train: {}".format(y_train.shape))

print("Shape of x_test: {}".format(x_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=25, validation_data=(x_test, y_test,verbose=2)